In [1]:
import re

from functools import lru_cache

from pymorphy2 import MorphAnalyzer
from nltk.stem import WordNetLemmatizer

ru_morph = MorphAnalyzer()
en_morph = WordNetLemmatizer()

CYRILLIC_PATTERN = re.compile('[а-яА-Я]')
DIGIT_PATTERN = re.compile('\d+')

def has_cyrillic(text):
    return bool(CYRILLIC_PATTERN.search(text))

def has_numeric(text):
    return bool(DIGIT_PATTERN.search(text))

@lru_cache(maxsize=1500)
def morph_process(token):
    if has_cyrillic(token):
        return ru_morph.parse(token)[0].normal_form
    elif has_numeric(token):
        return token
    else:
        return en_morph.lemmatize(token)
    
def morph_sentence(sentence):
    res = re.findall('\d+', sentence.lower())
    words = re.findall('[^\W\d_]+', sentence.lower())
    res.extend(map(morph_process, words))
    return res

In [2]:
s_expert = '"Русгидро", "Росгеология", "Транснефть" получат предложения перенести свои главные офисы на Дальний Восток'
s_system_1 = 'Компаниям «Русгидро», «Транснефть» и «Росгеология» предложили подумать о переезде на Дальний Восток.'
s_system_2 = 'По словам вице-премьера по Дальнему Востоку переезд может как-то затронуть "РусГидро", "Транснефть" и "Росгеологию"'

In [3]:
s_expert = morph_sentence(s_expert)
s_system_1 =  morph_sentence(s_system_1)
s_system_2 =  morph_sentence(s_system_2)

In [4]:
s_expert_ = set(s_expert)
s_system_1_ = set(s_system_1)
s_system_2_ = set(s_system_2)

rouge1_1 = len(s_expert_ & s_system_1_) / len(s_expert_)
rouge1_2 = len(s_expert_ & s_system_2_) / len(s_expert_)
rouge1_1, rouge1_2

(0.5, 0.4166666666666667)

In [5]:
s_expert_ = set(zip(s_expert[:-1], s_expert[1:]))
s_system_1_ = set(zip(s_system_1[:-1], s_system_1[1:]))
s_system_2_ = set(zip(s_system_2[:-1], s_system_2[1:]))

rouge2_1 = len(s_expert_ & s_system_1_) / len(s_expert_)
rouge2_2 = len(s_expert_ & s_system_2_) / len(s_expert_)
rouge2_1, rouge2_2

(0.18181818181818182, 0.09090909090909091)